In [0]:
# Defining the required source path

stage_table = 'event_catalog.orders_schema.stage_orders_table' 
final_table = 'event_catalog.orders_schema.orders_table' # Target table

In [0]:
# Reading the table and storing it in the dataframe
stage_df = spark.read.table(stage_table)


In [0]:
# importing the delta table library
from delta.tables import *

# Create the final table for the first time if that doesn't exist

if not spark._jsparkSession.catalog().tableExists(final_table):
    stage_df.write.format("delta").saveAsTable(final_table)
    
else:
    # Perform delta table merge query for upsert based on tracking_num column
    target_table = DeltaTable.forName(spark, final_table)

    # Define the merge condition based on the tracking_num column
    merge_condition = "stage.tracking_num = target.tracking_num"

    # Execute the merge operation
    target_table.alias("target") \
        .merge(stage_df.alias("stage"), merge_condition) \
        .whenMatchedDelete() \
        .execute()
    # if matched deleting the specific recored

    # Now, adding the new records along with the one's which deleted, to the undeleted records in the final table
    stage_df.write.format("delta").mode("append").saveAsTable(final_table)